In [2]:
import numpy as np
import tensorflow as tf
import joblib
from tqdm import tqdm
from src.rowwise_metrics import rowwise_cosine, rowwise_euclid
from src.metrics import rse

# dataset variables
from src.visualization import IndexType
INDEX = IndexType.HORIZONTAL_SNAKE

TRAIN_DIM = (560, 560)
VAL_DIM = (266, 500)
TEST_DIM = (500, 500)

s1_wavelengths = np.load(open('datasets/s1_wavelengths.npy', 'rb'))
s2_wavelengths = np.load(open('datasets/s2_wavelengths.npy', 'rb'))

# data and preprocessing

In [3]:
s1_train = np.load(open(f'datasets/s1_train.npy', 'rb'))[:20]
s2_train = np.load(open(f'datasets/s2_train.npy', 'rb'))[:20]
s1_val = np.load(open(f'datasets/s1_val.npy', 'rb'))[:20]
s2_val = np.load(open(f'datasets/s2_val.npy', 'rb'))[:20]
s1_test = np.load(open(f'datasets/s1_test.npy', 'rb'))[:20]
s2_test = np.load(open(f'datasets/s2_test.npy', 'rb'))[:20]

In [24]:
np.load(open(f'datasets/s2_test.npy', 'rb'))[:20].shape

(20, 3872)

In [4]:
from sklearn.preprocessing import StandardScaler
from src.preprocessing import correct_baseline

def preprocess_dataset(dataset):
    dataset = np.apply_along_axis(lambda x: correct_baseline(x, ww_min=100), 1, dataset)
    pipeline = StandardScaler().fit(dataset)
    dataset = pipeline.transform(dataset)
    return dataset, pipeline

In [5]:
s1_train, s1_train_pipeline = preprocess_dataset(s1_train)
s2_train, s2_train_pipeline = preprocess_dataset(s2_train)
s1_val,   s1_val_pipeline   = preprocess_dataset(s1_val)
s2_val,   s2_val_pipeline   = preprocess_dataset(s2_val)
s1_test,  s1_test_pipeline  = preprocess_dataset(s1_test)
s2_test,  s2_test_pipeline  = preprocess_dataset(s2_test)

# data exploration

In [6]:
from src.visualization import intensity_map, plot_spectra
from src.preprocessing import match_wavelengths
from plotly.express import colors

def hyperspectral_image(dataset, pipeline, dim, wavelength_from=None, wavelength_to=None):
    fig = intensity_map(pipeline.inverse_transform(dataset), dim=dim, index_type=INDEX, start=wavelength_from, end=wavelength_to)
    fig.update_layout(yaxis=dict(scaleanchor='x'))
    return fig


def s1_vs_s2_spectra(s1_spectrum, s2_spectrum, s1_pipeline, s2_pipeline, **kwargs):
    s1_spectrum, s2_spectrum, calibration = match_wavelengths(
        s1_pipeline.inverse_transform([s1_spectrum]),
        s2_pipeline.inverse_transform([s2_spectrum]),
        s1_wavelengths,
        s2_wavelengths,
    )
    fig = plot_spectra(
        np.vstack((s1_spectrum, s2_spectrum)),
        calibration=calibration,
        labels=['s1 spectrum', 's2 spectrum'],
        **kwargs
    )
    return fig

In [7]:
hyperspectral_image(s1_test, s1_test_pipeline, TEST_DIM).show()

In [8]:
id = 5

s1_vs_s2_spectra(
    s1_test[id],
    s2_test[id],
    s1_test_pipeline,
    s2_test_pipeline,
    title=f'spectra at id = {id} of MASTER test (s1 spectrum) and SUBORDINATE test (s2 spectrum) datasets'
).show()

del id

In [9]:
s1_vs_s2_spectra(
    s1_test.mean(axis=0),
    s2_test.mean(axis=0),
    s1_test_pipeline,
    s2_test_pipeline,
    title = 'mean spectra from the MASTER train and MASTER test datasets after preprocessing'    
).show()

In [10]:
plot_spectra(
    np.vstack((
        s1_train_pipeline.inverse_transform([s1_train.mean(axis=0)]),
        s1_test_pipeline.inverse_transform([s1_test.mean(axis=0)]),
    )),
    calibration=s1_wavelengths,
    labels=['train spectrum', 'test_spectrum'],
    colormap=colors.qualitative.Set1,
)

# model definition and training

## knn baseline (knn)

In [11]:
from sklearn.neighbors import KNeighborsRegressor

neighbors = [5, 7, 10, 12, 15]
cosines = []
mses = []
rses = []

knn = None
best_k = -1
best_score = -1

# optimize k
for n_neighbors in tqdm(neighbors):
    new = KNeighborsRegressor(n_neighbors=n_neighbors).fit(s2_train, s1_train)
    predicted = new.predict(s2_val)
    cosines.append(rowwise_cosine(s1_val, predicted).mean())
    mses.append(rowwise_euclid(s1_val, predicted).mean())
    rses.append(rse(s1_val, predicted))
    if mses[-1] > best_score:
        best_k = n_neighbors
        knn = new
    del new, predicted

print(
f'K was optimized to minimize the average euclidean distance on the validation dataset.\n\
Best k: {best_k}.\n\n\
    All scores:\n\
    -------------------------------------------'
)

for k, c, e, r in zip(neighbors, cosines, mses, rses):
    print(f'k: {k}, euclid: {e}, cosine: {c}, rse: {r}')

100%|██████████| 5/5 [00:00<00:00, 87.72it/s]

K was optimized to minimize the average euclidean distance on the validation dataset.
    best k: 15.

    All scores:
    -------------------------------------------
k: 5, euclid: 59.46232906422021, cosine: -0.3614647990411758, rse: 0.8710412751024782
k: 7, euclid: 61.098822803297615, cosine: -0.2851201672969111, rse: 0.9196432704555477
k: 10, euclid: 62.18600629142594, cosine: -0.21803197052993345, rse: 0.9525947838072949
k: 12, euclid: 62.642645815879334, cosine: -0.18289274643038614, rse: 0.9666967787760948
k: 15, euclid: 63.13216323937314, cosine: -0.13497298959158616, rse: 0.9818921486404092


## mean baseline (dummy)

In [12]:
from sklearn.dummy import DummyRegressor

dummy = DummyRegressor(strategy='mean').fit(s2_train, s1_train)

## proposed model (model)

### autoencoder (vae)

In [13]:
from src.keras_mixins import AnnealingCallback, Sampling, VariationalAutoencoder

def build_model(hp, regularizer=tf.keras.regularizers.l2(l2=1e-6)):
    neuron_counts = [
                    hp.Choice('first_layer', values=[2048, 1024, 128]),
                    hp.Choice('second_layer', values=[512, 64, 32]),
                    hp.Choice('bottleneck', values=[8, 32, 64]),
    ]
    neuron_counts.append(neuron_counts[1])
    neuron_counts.append(neuron_counts[0])
    learning_rate = hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])
    bottleneck = neuron_counts.index(min(neuron_counts))
    bottleneck_size = neuron_counts[bottleneck]

    # encoder
    inputs = tf.keras.Input(shape=s1_wavelengths.shape)
    x = tf.keras.layers.Dense(neuron_counts[0], activation='leaky_relu', kernel_regularizer=regularizer)(inputs)
    for nodes in neuron_counts[1:bottleneck]:
        x = tf.keras.layers.Dense(nodes, activation='leaky_relu', kernel_regularizer=regularizer)(x)
    z_mean = tf.keras.layers.Dense(bottleneck_size)(x)
    z_log_sigma = tf.keras.layers.Dense(bottleneck_size)(x)
    z = Sampling()([z_mean, z_log_sigma])
    encoder = tf.keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')
    encoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='mse')

    # decoder
    latent_inputs = tf.keras.Input(shape=(bottleneck_size,), name='z_sampling')
    x = tf.keras.layers.Dense(neuron_counts[bottleneck + 1], activation='leaky_relu', kernel_regularizer=regularizer)(latent_inputs)
    for nodes in neuron_counts[bottleneck + 2:]:
        x = tf.keras.layers.Dense(nodes, activation='leaky_relu', kernel_regularizer=regularizer)(x)
    outputs = tf.keras.layers.Dense(s1_wavelengths.shape[0])(x)
    decoder = tf.keras.Model(latent_inputs, outputs, name='decoder')
    decoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='mse')

    outputs = decoder(encoder(inputs)[2])
    vae = tf.keras.Model(inputs, outputs, name='vae_mlp')

    vae = VariationalAutoencoder(encoder, decoder)
    vae.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='mse')

    return vae

In [14]:
import keras_tuner

epochs = 50
batch_size = 128

tuner = keras_tuner.Hyperband(build_model,
                    objective='val_loss',
                    max_epochs=50,
                    #overwrite = True,
                    )
tuner.search(
    s1_train, epochs=epochs, batch_size=batch_size, validation_data=(s1_val,),
    verbose=2, callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_reconstruction_loss', patience=5, min_delta=1e-4,),
        AnnealingCallback(epochs),
    ]
)

vae = tuner.get_best_models()[0]
# restore weights
vae.predict(s1_train[:5])

del epochs, batch_size, tuner

Trial 90 Complete [00h 00m 19s]
val_loss: 1.0212492942810059

Best val_loss So Far: 1.000724196434021
Total elapsed time: 00h 12m 27s
INFO:tensorflow:Oracle triggered exit
1/1 [==============================] - 0s 378ms/step


### mlp (insert)

In [15]:
bottleneck_size = vae.get_layer('encoder').layers[-1].output_shape[-1]

def build_model(hp, regularizer=tf.keras.regularizers.l2(l2=1e-6)):
    neuron_counts = [
                    hp.Choice('first_layer', values=[2048, 1024, 512, 128]),
                    hp.Choice('second_layer', values=[1024, 512, 128]),
    ]
    learning_rate = hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])
    input = tf.keras.layers.Input(shape=s2_wavelengths.shape, name='insert_in')
    x = tf.keras.layers.Dense(neuron_counts[0], kernel_regularizer=regularizer, activation='leaky_relu')(input)
    for n in neuron_counts[1:]:
        x = tf.keras.layers.Dense(n, kernel_regularizer=regularizer, activation='leaky_relu')(x)
    x = tf.keras.layers.Dense(bottleneck_size, name='output')(x)

    insert = tf.keras.Model(input, x, name='Inserter')
    insert.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))

    return insert

In [16]:
batch_size = 128
epochs = 100

train_latent = vae.encode(s1_train)
val_latent = vae.encode(s1_val)
tuner = keras_tuner.Hyperband(build_model,
                    objective='val_loss',
                    max_epochs=50,
                    overwrite = True,
                    )
tuner.search(
    s2_train, train_latent, validation_data=(s2_val, val_latent), epochs=epochs,
    batch_size=batch_size, verbose=2,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, min_delta=1e-5,)]
)
del train_latent, val_latent

insert = tuner.get_best_models()[0]
#restore weights
insert.predict(s2_train[:5])

del epochs, batch_size, tuner


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
1024              |?                 |first_layer
512               |?                 |second_layer
1e-05             |?                 |learning_rate
2                 |?                 |tuner/epochs
0                 |?                 |tuner/initial_epoch
3                 |?                 |tuner/bracket
0                 |?                 |tuner/round

Epoch 1/2


ValueError: in user code:

    File "c:\Users\Pavel\Desktop\git\LIBS_library\.venv\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Pavel\Desktop\git\LIBS_library\.venv\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Pavel\Desktop\git\LIBS_library\.venv\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\Pavel\Desktop\git\LIBS_library\.venv\lib\site-packages\keras\engine\training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\Pavel\Desktop\git\LIBS_library\.venv\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\Pavel\Desktop\git\LIBS_library\.venv\lib\site-pack

In [20]:
print(s2_wavelengths.shape[0])

s2_val.shape[1]

3872


4062

### combined model (model)

In [ ]:
learning_rate = 1e-6
metrics=[tf.keras.losses.cosine_similarity]

decoder = vae.decoder
input = tf.keras.layers.Input(shape=s2_wavelengths.shape, name='model_in')
model_encoder = tf.keras.models.clone_model(insert)
model_encoder.set_weights(insert.get_weights())
model_decoder = tf.keras.models.clone_model(decoder)
model_decoder.set_weights(decoder.get_weights())
encoded = model_encoder(input)
decoded = model_decoder(encoded)
model = tf.keras.Model(input, decoded, name="Model")
model_encoder.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))
model_decoder.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))
model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), metrics=metrics)

del learning_rate, metrics, input, model_encoder, model_decoder, encoded, decoded

## mlp baseline (mlp_baseline)

In [ ]:
learning_rate = 1e-4
metrics=[tf.keras.losses.cosine_similarity]

input = tf.keras.layers.Input(shape=s2_wavelengths.shape, name='baseline_in')
mlp_encoder = tf.keras.models.clone_model(insert)
mlp_decoder = tf.keras.models.clone_model(decoder)
encoded = model_encoder(input)
decoded = model_decoder(encoded)
mlp_baseline = tf.keras.Model(input, decoded, name="Model")
mlp_encoder.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))
mlp_decoder.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))
mlp_baseline.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), metrics=metrics)

del learning_rate, metrics, input, mlp_encoder, mlp_decoder, encoded, decoded

In [ ]:
epochs = 100
batch_size = 128

mlp_baseline.fit(
    s2_train, s1_train,
    validation_data=(s2_val, s1_val),
    epochs=epochs,
    batch_size=batch_size,
    verbose=2,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, min_delta=1e-5)]
)

del epochs, batch_size

# evaluation

In [ ]:
# expected reduction in size of shared data
bottleneck = 64
print(1 - (bottleneck * TEST_DIM[0] * TEST_DIM[1] + model.get_layer('decoder').count_params()) / (s1_wavelengths.shape[0] * TEST_DIM[0] * TEST_DIM[1]))

In [ ]:
from src.visualization import error_map
from src.rowwise_metrics import rowwise_euclid, rowwise_cosine
from src.metrics import rse

def evaluate_model(model, s2_dataset, s1_dataset, s1_pipeline):
    predicted = s1_pipeline.iverse_transform(model.predict(s2_dataset))
    original = s1_pipeline.inverse_transform(s1_dataset)

    print(
    f"    Numeric results\n\
        -------------------------------------------\n\
    RSE                        : {rse(original, predicted)}\n\
    average Euclidean distance : {rowwise_euclid(original, predicted).mean()}\n\
    average cosine distance    : {rowwise_cosine(original, predicted).mean()}\n"
    )
    return original, predicted

In [ ]:
dim = TEST_DIM
original, predicted = evaluate_model(model, s2_test, s1_test, s1_test_pipeline)

In [ ]:
error_map(
    y_true=original, y_pred=predicted,
    dim=dim,
    index_type=INDEX,
    rowwise_error=rowwise_euclid,
    title='Euclidean distance map',
    add_stats=True,
).show()

In [ ]:
error_map(
    y_true=original, y_pred=predicted,
    dim=dim,
    index_type=INDEX,
    rowwise_error=rowwise_cosine,
    title='Cosine distance map',
    add_stats=True,
).show()

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from src.visualization import plot_map

# clustering experiment

## optimize k
ks = range(3, 12)
scores = []
best_score = -1
best_k = -1
kmeans = None
for n_clusters in tqdm(ks):
    new = KMeans(n_clusters=n_clusters, random_state=42).fit(original)
    score = silhouette_score(
        original, new.predict(original),
        random_state=42, sample_size=25000
    )
    scores.append(score)
    if score > best_score:
        best_score = score
        best_k = n_clusters
        kmeans = new

print(
f'K was optimized to minimize the silhouette score on the evaluation (originally set to test) dataset.\n\
Best k: {best_k}.\n\n\
    All scores:\n\
    -------------------------------------------'
)
for k, score in zip(ks, scores):
    print(f'k: {k}, silhouette score: {score}')


## k-score
original_labels = kmeans.predict(original)
predicted_labels = kmeans.predict(predicted.astype(float))

plot_map(
    values=original_labels,
    dim=dim,
    index_type=INDEX,
    title='labels assigned to original spectra',
).show()

plot_map(
    values = 1 - np.equal(predicted_labels, original_labels).astype(int),
    dim=dim,
    index_type=INDEX,
    title='predicted spectra with different label than corresponding original',
).show()

In [ ]:
# samples

x, y = 0, 0

def id_from_snake_index(x, y, dim):
    return y * dim[1] + (x if not y % 2 else dim[1] - x - 1)

id = id_from_snake_index(x, y, dim)

plot_spectra(
    np.concatenate((original[id], predicted[id])),
    calibration=s1_wavelengths,
    title=f'Sample at x, y coordinates: ({x}, {y})',
    labels=['original', 'predicted']
).show()